# Even / Odd training denoising of epfl1_subset1_float

* Inputs: `even.mrc` and `odd.mrc`, the pair of noisy vols.
* Outputs: `even_odd_denoised/epfl1_subset1_float.mrc` with the denoised vol, and `denoise_even_odd.pdf` with a view of a tile of a central slice in Z.

In [ ]:
from pathlib import Path
from collections import namedtuple

In [ ]:
Args = Args = namedtuple("args", ["even", "odd", "denoised"])
args = Args("even.mrc", "odd.mrc", "even_odd_denoised/epfl1_subset1_float.mrc")

In [ ]:
if Path(args.denoised).exists():
    raise Exception(f"{args.denoised} already exists ... exiting")

In [ ]:
if not Path("even.mrc").exists() or not Path("odd.mrc"):
    %run split_even_odd.ipynb

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'

## Configure cryoCARE

In [ ]:
%%writefile train_data_config__evenodd.json
{
    "even": ["even.mrc"],
    "odd": ["odd.mrc"],
    "mask": [""],
    "patch_shape": [32, 32, 32],
    "num_slices": 360,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./even_odd_data",
    "overwrite": "True"  
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/epfl1
source ~/envs/cryoCARE/bin/activate
cryoCARE_extract_train_data.py --conf train_data_config__evenodd.json

## Train

In [ ]:
%%writefile train_config__evenodd.json
{
  "train_data": "./even_odd_data",
  "epochs": 50,
  "steps_per_epoch": 200,
  "batch_size": 16,
  "unet_kern_size": 3,
  "unet_n_depth": 3,
  "unet_n_first": 16,
  "learning_rate": 0.0004,
  "model_name": "model",
  "path": "./",
  "gpu_id": [0, 1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/epfl1
source ~/envs/cryoCARE/bin/activate
cryoCARE_train.py --conf train_config__evenodd.json

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='model.tar.gz',
    drive_file_name='epfl1_subset1_float__denoised__cryoCARE_even_odd__model.tar.gz',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

## Infer

In [ ]:
!ln -sf /nas/vruiz/vols/epfl1_subset1_float.mrc .

In [ ]:
%%writefile predict_config__evenodd.json
{
    "path": "./model.tar.gz",
    "even": ["epfl1_subset1_float.mrc"], 
    "odd": ["epfl1_subset1_float.mrc"],
    "n_tiles": [1,1,1],
    "output": "even_odd_denoised",
    "overwrite": "True",
    "gpu_id": [0, 1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/epfl1
pwd
source ~/envs/cryoCARE/bin/activate
cryoCARE_predict.py --conf predict_config__evenodd.json || true

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='even_odd_denoised/epfl1_subset1_float.mrc',
    drive_file_name='epfl1_subset1_float__denoised__cryoCARE_even_odd.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
import mrcfile
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
def read_MRC(path_file):
    return mrcfile.read(mrc_file_path)

In [ ]:
mrc_file_path = '/nas/vruiz/cryoCARE/epfl1/epfl1_subset1_float.mrc'
original_volume = read_MRC(mrc_file_path)

In [ ]:
original_volume.shape

In [ ]:
mrc_file_path = '/nas/vruiz/cryoCARE/epfl1/even_odd_denoised/epfl1_subset1_float.mrc'
denoised_volume = read_MRC(mrc_file_path)

In [ ]:
denoised_volume.shape

In [ ]:
# Choose a slice index in the middle of the volume for a good comparison
slice_idx = original_volume.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N Even/Odd Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised_volume.shape[0]//2
plt.imshow(denoised_volume[slice_idx, 300:, 600:], cmap="gray")
plt.savefig("denoise_even_odd.pdf")

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoise_even_odd.pdf',
    drive_file_name='epfl1_subset1_float__denoised__cryoCARE_even_odd.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
%pip install "self_fourier_shell_correlation @ git+https://github.com/vicente-gonzalez-ruiz/self_fourier_shell_correlation"

In [ ]:
%pip show self_fourier_shell_correlation

In [ ]:
import sys
print(sys.executable)

In [ ]:
%pip install "shuffling @ git+https://github.com/vicente-gonzalez-ruiz/shuffling"

In [ ]:
%pip install opencv-python

In [ ]:
%pip install "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
from self_fourier_shell_correlation import fsc_utils as fsc

In [ ]:
import mrcfile

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
even = mrcfile.read("even.mrc")

In [ ]:
odd = mrcfile.read("odd.mrc")

In [ ]:
import numpy as np
from scipy.fft import fftn

In [ ]:
def compute_spatial_frequencies(shape):
    """
    Compute the spatial frequency grid for a volume of arbitrary shape.
    
    Args:
        shape: Tuple representing the shape of the volume (nx, ny, nz).
        
    Returns:
        freq_radii: A 3D array where each element represents the spatial frequency radius at that point.
    """
    freq_x = np.fft.fftfreq(shape[0])
    freq_y = np.fft.fftfreq(shape[1])
    freq_z = np.fft.fftfreq(shape[2])

    freq_x, freq_y, freq_z = np.meshgrid(freq_x, freq_y, freq_z, indexing='ij')

    freq_radii = np.sqrt(freq_x**2 + freq_y**2 + freq_z**2)

    return freq_radii

def fourier_shell_correlation(volume1, volume2, shell_thickness=1):
    """
    Compute the Fourier Shell Correlation (FSC) between two volumes of arbitrary shape.
    Args:
        volume1, volume2: Two 3D volumes to compare
        shell_thickness: Thickness of Fourier shells in frequency units
    Returns:
        spatial_freq: Array of spatial frequencies (1/voxel units)
        fsc_values: Array of FSC values at each spatial frequency
    """
    # Fourier transforms of both volumes (in double precision and normalized)
    fft1 = fftn(volume1).astype(np.complex128) / np.sqrt(np.prod(volume1.shape))
    fft2 = fftn(volume2).astype(np.complex128) / np.sqrt(np.prod(volume2.shape))

    # Compute spatial frequency grid based on the volume shape
    freq_radii = compute_spatial_frequencies(volume1.shape)

    max_radius = np.max(freq_radii)
    shell_indices = np.arange(0, max_radius, shell_thickness)

    fsc_values = []
    spatial_freq = []

    for r in shell_indices:
        shell_mask = (freq_radii >= r) & (freq_radii < r + shell_thickness)
        
        # Numerator and denominator of the FSC
        num = np.sum(fft1[shell_mask] * np.conj(fft2[shell_mask]))
        denom = np.sqrt(np.sum(np.abs(fft1[shell_mask])**2) * np.sum(np.abs(fft2[shell_mask])**2))
        
        # Handle potential division by zero
        fsc_value = np.abs(num) / denom if denom != 0 else 0
        fsc_values.append(fsc_value)
        spatial_freq.append(r)

    return np.array(spatial_freq), np.array(fsc_values)

In [ ]:
spatial_freq, fsc_values = fsc.fourier_shell_correlation(even, odd, shell_thickness=0.01)

In [ ]:
fsc.plot_fsc(spatial_freq, fsc_values, "Spatial Frequency (1/voxel units)", "FSC", "FSC(even, odd)")